In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# p.176 必要なライブラリをインポートする
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

# p.178 ランダムシードの設定
import random
np.random.seed(1234)
random.seed(1234)

# p.184 LabelEncoderのライブラリをインポート
from sklearn.preprocessing import LabelEncoder

# p.188 LightGBMのライブラリをインポート
import lightgbm as lgb

# リスト4.17 LightGBMのハイパーパラメータを設定
lgbm_params = {
    "objective":"regression",
    "random_seed":1234
}

# p.189 クロスバリデーション用のライブラリを読み込んで分割数を3に設定
from sklearn.model_selection import KFold
folds = 3
kf = KFold(n_splits=folds)

# p.190 リスト4.19 平均二乗誤差を出すライブラリをインポート
from sklearn.metrics import mean_squared_error

In [5]:
# p.179 CSVデータを読み込む
train_df = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
submission = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")

# train_df.head()

# p.180 学習データの各変数の型を確認する
# train_df.dtypes

# p.181 MSZoningの各分類ごとの個数を確認する
# train_df["MSZoning"].value_counts()

# p.182 学習データとテストデータの連結
all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
# all_df

# p.183 目的変数であるSalePriceの値を確認
# all_df["SalePrice"]

# object型の変数を取得
#categories = all_df.columns[all_df.dtypes == "object"]
# print(categories)

# p.185 'Alley'の各分類の個数を確認
# all_df["Alley"].value_counts()

# 欠損値を数値に変換する
for cat in categories:
    le = LabelEncoder()
    # print(cat)
    all_df[cat].fillna("missing", inplace=True)
    le = le.fit(all_df[cat])
    all_df[cat] = le.transform(all_df[cat])
    all_df[cat] = all_df[cat].astype("category")
# all_df

# p.188 データをtrain_dfとtest_dfに戻す
#train_df_le = all_df[~all_df["SalePrice"].isnull()]
#test_df_le = all_df[all_df["SalePrice"].isnull()]
# train_df_le.head()
# test_df_le.head()

# リスト4.18 説明変数、目的変数を指定
#train_X = train_df_le.drop(["SalePrice", "Id"], axis=1)
#train_Y = train_df_le["SalePrice"]

# p.190 リスト4.20 各foldごとに作成したモデルごとの予測値を保存
#models = []
#rmses = []
#oof = np.zeros(len(train_X))

#for train_index, val_index in kf.split(train_X):
#    X_train = train_X.iloc[train_index]
#    X_valid = train_X.iloc[val_index]
#    y_train = train_Y.iloc[train_index]
#    y_valid = train_Y.iloc[val_index]
    
#    lgb_train = lgb.Dataset(X_train, y_train)
#    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
#    model_lgb = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval, num_boost_round=100, early_stopping_rounds=20, verbose_eval=10,)
    
#    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
#    tmp_rmse = np.sqrt(mean_squared_error(np.log(y_valid), np.log(y_pred)))
#    print(tmp_rmse)
    
#    models.append(model_lgb)
#    rmses.append(tmp_rmse)
#    oof[val_index] = y_pred
    
# p.192 平均RMSEを計算する
# sum(rmses)/len(rmses)

# p.193 statisticsライブラリから計算
#from statistics import mean
#mean(rmses)

# p.193 現状の予測値と実際の値の違いを可視化
#actual_pred_df = pd.DataFrame({"actual": train_Y, "pred": oof})
#actual_pred_df.plot(figsize=(12,5))

# p.194 各変数の重要度を確認する
# リスト4.24 変数の数を制限して各変数の重要度を表示
#for model in models:
#    lgb.plot_importance(model, importance_type="gain", max_num_features=15)

# p.197 リスト4.25 SalePriceの各統計量を確認する
#train_df["SalePrice"].describe()

# p.198 リスト4.26 ヒストグラムで分布を確認
#train_df["SalePrice"].plot.hist(bins=20)

# p.199 リスト4.27 SalePriceを対数化
#np.log(train_df['SalePrice'])

# 対数化したSalePriceの分布をヒストグラムで可視化
#np.log(train_df['SalePrice']).plot.hist(bins=20)

# p.200 リスト4.29 対数化による予測精度の向上を確認
#train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
#train_X = train_df_le.drop(["SalePrice", "SalePrice_log", "Id"], axis=1)
#train_Y = train_df_le["SalePrice_log"]

#models = []
#rmses = []
#oof = np.zeros(len(train_X))

#for train_index, val_index in kf.split(train_X):
#    X_train = train_X.iloc[train_index]
#    X_valid = train_X.iloc[val_index]
#    y_train = train_Y.iloc[train_index]
#    y_valid = train_Y.iloc[val_index]
    
#    lgb_train = lgb.Dataset(X_train, y_train)
#    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    
#    model_lgb = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval, num_boost_round=100, early_stopping_rounds=20, verbose_eval=10,)
    
#    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
#    tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
#    print(tmp_rmse)
    
#    models.append(model_lgb)
#    rmses.append(tmp_rmse)
#    oof[val_index] = y_pred
    
#sum(rmses)/len(rmses)

# p.204 リスト4.30 all_dfの作成
all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
categories = all_df.columns[all_df.dtypes == "object"]
#print(categories)

# p.205 リスト4.31 欠損値の数が上位40の変数を確認
#all_df.isnull().sum().sort_values(ascending=False).head(40)

# p.207 リスト4.32 PoolQCの各分類ごとの個数
#all_df.PoolQC.value_counts()

# リスト4.33 PoolQCの値を値があるものを1、値がないものを0に変換
#all_df.loc[~all_df["PoolQC"].isnull(), "PoolQC"] = 1
#all_df.loc[all_df["PoolQC"].isnull(), "PoolQC"] = 0
# all_df.PoolQC.value_counts()

# p.208 リスト4.35 MiscFeature, Alleyも0と1に変換する
#all_df.loc[~all_df["MiscFeature"].isnull(), "MiscFeature"] = 1
#all_df.loc[all_df["MiscFeature"].isnull(), "MiscFeature"] = 0
#all_df.loc[~all_df["Alley"].isnull(), "Alley"] = 1
#all_df.loc[~all_df["Alley"].isnull(), "Alley"] = 0

# リスト4.36 繰り返し処理はfor文でまとめる
HighFacility_col = ["PoolQC", "MiscFeature", "Alley"]
for col in HighFacility_col:
    if all_df[col].dtype == "object":
        if len(all_df[all_df[col].isnull()]) > 0:
            all_df.loc[~all_df[col].isnull(), col] = 1
            all_df.loc[all_df[col].isnull(), col] = 0

# p.209 リスト4.37 0か1の値に変換した各変数を足し合わせて、高級住宅設備の数という特徴量を作成
all_df["hasHighFacility"] = all_df["PoolQC"] + all_df["MiscFeature"] + all_df["Alley"]
all_df["hasHighFacility"] = all_df["hasHighFacility"].astype(int)
# all_df.hasHighFacility.value_counts()

# p.209 リスト4.39 もとのデータからPoolQC, MiscFeature, Alleyを削除
all_df = all_df.drop(["PoolQC", "MiscFeature", "Alley"], axis=1)

# p.211 リスト4.40 各変数の統計量を確認する
#all_df.describe().T # 転置

# p.212 リスト4.41 数値データのみの抜き出し
train_df_num = train_df.select_dtypes(include=[np.number])

# リスト4.42 比例尺度ではない変数
nonratio_features = ["Id", "MSSubClass", "OverallQual", "OverallCond", "YearRemodAdd", "MoSold", "YrSold"]

# リスト4.43 数値データからリスト4.43の変数を除いた比例尺度データ
num_features = sorted(list(set(train_df_num) - set(nonratio_features)))
#num_features

# p.213 リスト4.44 比例尺度の列のみを抜き出す
train_df_num_rs = train_df_num[num_features]

# p.214 リスト4.45 3/4分位数が0となる変数を確認
#for col in num_features:
#    if train_df_num_rs.describe()[col]["75%"] == 0:
#        print(col, len(train_df_num_rs[train_df_num_rs[col] == 0]))

# p.215 リスト4.46 ある特定の値のみしかとらないものを確認
#for col in num_features:
#    if train_df_num_rs[col].nunique() < 15:
#        print(col, train_df_num_rs[col].nunique())
        
# リスト4.47 外れ値があるか確認
for col in num_features:
    tmp_df = train_df_num_rs[(train_df_num_rs[col] > train_df_num_rs[col].mean() + train_df_num_rs[col].std()*3) | (train_df_num_rs[col] < train_df_num_rs[col].mean() - train_df_num_rs[col].std()*3)]
#    print(col, len(tmp_df))
 
# p.217 リスト4.48 BsmtFinSF1とSalePriceの分布を可視化
#all_df.plot.scatter(x="BsmtFinSF1", y="SalePrice")

# リスト4.49 BsmtFinSF1が広いもののSalePriceが高くないものを確認
#all_df[all_df["BsmtFinSF1"] > 5000]

# p.218 TotalBsmtSFとSalePriceの分布を可視化
# all_df.plot.scatter(x="TotalBsmtSF", y="SalePrice")
# all_df[all_df["TotalBsmtSF"] > 6000]

# p.219 GrLivAreaとSalePriceの分布を可視化
#all_df.plot.scatter(x="GrLivArea", y="SalePrice")
#all_df[all_df["GrLivArea"] > 5000]

# p.219 1stFlrSFとSalePriceの分布を可視化
#all_df.plot.scatter(x="1stFlrSF", y="SalePrice")
#all_df[all_df["1stFlrSF"] > 4000]

# p.221 リスト4.53 外れ値以外を抽出（テストデータはすべて抽出）
all_df = all_df[(all_df['BsmtFinSF1'] < 2000) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['TotalBsmtSF'] < 3000) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['GrLivArea'] < 4500) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['1stFlrSF'] < 2500) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['LotArea'] < 100000) | (all_df['SalePrice'].isnull())]

# リスト4.54 categoriesの中から除外した3つの変数を削除
categories = categories.drop(["PoolQC", "MiscFeature", "Alley"])

# リスト4.55 欠損値をmissingに置き換えてall_dfのカテゴリ変数をcategoryに設定
for cat in categories:
    le = LabelEncoder()
    print(cat)
    
    all_df[cat].fillna("missing", inplace=True)
    le = le.fit(all_df[cat])
    all_df[cat] = le.transform(all_df[cat])
    all_df[cat] = all_df[cat].astype("category")

# p.223 リスト4.56 train_df_leとtest_df_leに分割
train_df_le = all_df[~all_df["SalePrice"].isnull()]
test_df_le = all_df[all_df["SalePrice"].isnull()]

train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
train_X = train_df_le.drop(["SalePrice", "SalePrice_log", "Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]

models = []
rmses = []
oof = np.zeros(len(train_X))

for train_index, val_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    X_valid = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_valid = train_Y.iloc[val_index]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    
    model_lgb = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval, num_boost_round=100, early_stopping_rounds=20, verbose_eval=10,)
    
    y_pred = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration)
    tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print(tmp_rmse)
    
    models.append(model_lgb)
    rmses.append(tmp_rmse)
    oof[val_index] = y_pred
    
    sum(rmses)/len(rmses)

MSZoning
Street
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
Fence
SaleType
SaleCondition
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000792 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2949
[LightGBM] [Info] Number of data points in the train set: 963, number of used features: 72


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecat

[LightGBM] [Info] Start training from score 12.010615
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l2: 0.0423077
[20]	valid_0's l2: 0.0220026
[30]	valid_0's l2: 0.0175682
[40]	valid_0's l2: 0.0163798
[50]	valid_0's l2: 0.0162547
[60]	valid_0's l2: 0.0162622
[70]	valid_0's l2: 0.0162514
Early stopping, best iteration is:
[58]	valid_0's l2: 0.0161791
0.12719701309461987


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in para

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001805 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2943
[LightGBM] [Info] Number of data points in the train set: 963, number of used features: 72
[LightGBM] [Info] Start training from score 12.018820
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l2: 0.0467163
[20]	valid_0's l2: 0.026693
[30]	valid_0's l2: 0.0218687
[40]	valid_0's l2: 0.0201122
[50]	valid_0's l2: 0.0196764
[60]	valid_0's l2: 0.0196786
[70]	valid_0's l2: 0.0196393
[80]	valid_0's l2: 0.0195357
[90]	valid_0's l2: 0.0194547
[100]	valid_0's l2: 0.0193847
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.0193847
0.13922901661972728
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001808 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2987
[LightGB

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in para

[50]	valid_0's l2: 0.0151884
[60]	valid_0's l2: 0.0151752
[70]	valid_0's l2: 0.0153067
Early stopping, best iteration is:
[58]	valid_0's l2: 0.0151274
0.12299343081439215
